# Wine Dataset KD Tree

In [1]:
from sklearn.datasets import load_wine
import pandas as pd
import copy

In [2]:
data = load_wine(as_frame=True)

In [3]:
wine_df = data.frame
wine_df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [4]:
features = wine_df.columns.tolist()
features.pop(13)
features

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [5]:
class Nodo:
    
    def __init__(self, data, feature):
      self.hijoIzq = None
      self.hijoDer = None
      self.feature = feature
      self.data = data
        
    def insert(self, nodo, esMenor):
      if self.data:
         if esMenor:
            if self.hijoIzq is None:
               self.hijoIzq = nodo
            else:
               self.hijoIzq.insert(nodo, esMenor)
         else:
            if self.hijoDer is None:
               self.hijoDer = nodo
            else:
               self.hijoDer.insert(nodo, esMenor)
      else:
         self.data = data
         self.feature = feature

In [6]:
class Trunk:
    def __init__(self, prev=None, str=None):
        self.prev = prev
        self.str = str
 
def showTrunks(p):
    if p is None:
        return
    showTrunks(p.prev)
    print(p.str, end='')
 
 
def printTree(root, prev, isLeft):
    if root is None:
        return
 
    prev_str = '    '
    trunk = Trunk(prev, prev_str)
    printTree(root.hijoDer, trunk, True)
 
    if prev is None:
        trunk.str = '———'
    elif isLeft:
        trunk.str = '.———'
        prev_str = '   |'
    else:
        trunk.str = '`———'
        prev.str = prev_str
 
    showTrunks(trunk)
    print(' ' + str(root.feature+" "+str(round(root.data,1))))
    if prev:
        prev.str = prev_str
    trunk.str = '   |'
    printTree(root.hijoIzq, trunk, False)

In [7]:
def is_unique(s):
    if (s.shape[0]==1):
        return True
    else:
        a = s.to_numpy()
        #print(a)
        return (a[0] == a).all()

In [8]:
def calcular_umbral(features,pos, df, string):
    feature = features[pos]
    df = df.sort_values(by=[feature])
    
    row=0
    if(is_unique(df[feature])==False):
        while(df[feature].iloc[[df.shape[0]//2 - (row+1)]].values[0]==df[feature].iloc[[df.shape[0]//2 - row]].values[0]):
            row=row-1

    umbral = ((df[feature].iloc[[df.shape[0]//2 - (row+1)]].values[0] + df[feature].iloc[[df.shape[0]//2 - row]].values[0])/2)

    print('Variable:',feature,'-',' Umbral:', umbral, '|'+string)
    left_df = df.iloc[0:df.shape[0]//2 - (row)]
    right_df = df.iloc[df.shape[0]//2 - row:df.shape[0]]
    print('________________________________')
    print('Left')
    print(left_df[[feature,'target']])
    print('________________________________')
    print('Right')
    print(right_df[[feature,'target']])
    print('________________________________')
    
    node = Nodo(umbral,feature)
    
    if(pos==12):
        pos=-1
    if(is_unique(left_df['target'])):
        print('Clasificacion menor al umbral: ',left_df['target'].unique())
        node.hijoIzq = Nodo(left_df['target'].unique()[0],'target')
    else:
        _,_,nodox=calcular_umbral(features,pos+1, left_df,string + ' MENOR AL UMBRAL| '+features[pos+1])
        node.insert(nodox,True)
    if(is_unique(right_df['target'])):
        print('Clasificacion mayor al umbral: ',right_df['target'].unique())
        node.hijoDer = Nodo(right_df['target'].unique()[0],'target')
    else:        
        _,_,nodoy=calcular_umbral(features,pos+1, right_df,string+ ' MAYOR AL UMBRAL| '+features[pos+1])
        node.insert(nodoy,False)
    return left_df,right_df, node

In [9]:
_,_,tree = calcular_umbral(features,0,wine_df,'RAIZ(alcohol)')

Variable: alcohol -  Umbral: 13.06 |RAIZ(alcohol)
________________________________
Left
     alcohol  target
115    11.03       1
113    11.41       1
120    11.45       1
110    11.46       1
121    11.56       1
..       ...     ...
44     13.05       0
25     13.05       0
83     13.05       1
37     13.05       0
50     13.05       0

[93 rows x 2 columns]
________________________________
Right
     alcohol  target
38     13.07       0
149    13.08       2
66     13.11       1
152    13.11       2
145    13.16       2
..       ...     ...
46     14.38       0
14     14.38       0
6      14.39       0
13     14.75       0
8      14.83       0

[85 rows x 2 columns]
________________________________
Variable: malic_acid -  Umbral: 1.79 |RAIZ(alcohol) MENOR AL UMBRAL| malic_acid
________________________________
Left
     malic_acid  target
113        0.74       1
84         0.89       1
76         0.90       1
80         0.92       1
59         0.94       1
85         0.98       1
78  

In [10]:
space = 5
height = 5
printTree(tree, None, False)

                                .——— target 0
                            .——— nonflavanoid_phenols 0.3
                           |    `——— target 2
                        .——— flavanoids 1.1
                       |    `——— target 2
                    .——— total_phenols 1.8
                   |    `——— target 2
                .——— magnesium 108.0
               |    `——— target 2
            .——— alcalinity_of_ash 20.5
           |   |    .——— target 0
           |    `——— magnesium 110.0
           |       |    .——— target 0
           |        `——— total_phenols 2.9
           |           |        .——— target 2
           |           |    .——— nonflavanoid_phenols 0.4
           |           |   |    `——— target 0
           |            `——— flavanoids 0.6
           |                `——— target 2
        .——— ash 2.5
       |   |    .——— target 2
       |    `——— alcalinity_of_ash 20.2
       |       |        .——— target 0
       |       |    .——— total_phenols 2.7
       |    

In [11]:
test_data = wine_df.iloc[:,0:13]
original_target= wine_df['target']
original_target = original_target.values.tolist()

In [12]:
def rewriteTest(tree, data, original_target):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    predicciones=[]
    for row in data.iterrows():
        tree_copy = copy.deepcopy(tree)
        #print(type(row))
        #print(row[1])
        for index, value in row[1].items():
          # print(f"Index : {index}, Value : {value}")
          # print(type(value))
          # print(type(tree_copy.data.tolist()))
            if (tree_copy.feature == 'target'):
                predicciones.append(tree_copy.data)
                break
            else:
                if(tree_copy.feature==index and value>tree_copy.data.tolist()):
                    tree_copy=tree_copy.hijoDer
                else:
                    tree_copy=tree_copy.hijoIzq
                    
    for t,p in zip(original_target, predicciones):
        if t == p:
           if p == 1:
               tp += 1
           else:
               tn += 1
        else:
           if p == 1:
               fn += 1
           else:
               fp += 1

    precision = tp / (tp + fp)
    exactitud = (tp + tn) / (tp + tn + fp + fn)
    
    return precision, exactitud, predicciones

In [13]:
precision,exactitud,predicciones=rewriteTest(tree,test_data,original_target)

In [14]:
print('prediccion: '+str(predicciones))
print('original: '+str(original_target))

prediccion: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
original: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

In [15]:
precision

0.9859154929577465

In [16]:
exactitud

0.9943820224719101